In [2]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.23.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 49.0 MB/s eta 0:00:0000:0100:01
Using cached huggingface_hub-0.23.0-py3-none-any.whl (401 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Unins

In [3]:
!pip install llama-cpp-python


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
!pip -q install git+https://github.com/huggingface/transformers

In [6]:
!pip install PyPDF2

In [4]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [8]:
!pip install pandas

In [9]:
import pandas as pd
import numpy as np

In [10]:
# Load the CSV file
csv_file_path = "298a/model_data.csv"
data = pd.read_csv(csv_file_path)
data

,SUBJECT_ID,HADM_ID,TEXT,readmitted
0,2,163353,neonatology attending triage notebaby name ni ...,0
1,3,145834,2101106 602 pm chest portable ap clip clip num...,0
2,4,185777,2191315 420 pm chest portable ap clip clip num...,0
3,5,178980,nnp triage notebb known lastname 6 delivered 4...,0
4,6,107064,2175525 1052 chest preop pa lat clip clip numb...,0
...,...,...,...,...
52949,99985,176670,sinus rhythm normal ecg since previous tracing...,0
52950,99991,151118,21841227 1135 chest pa lat clip clip number ra...,0
52951,99992,197084,2144710 804 chest port line placement clip cli...,0
52952,99995,137810,2147110 815 carotid series complete clip clip ...,0


In [11]:
data.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=True)

In [12]:
data.drop('HADM_ID',axis=1)

,SUBJECT_ID,TEXT,readmitted
0,2,neonatology attending triage notebaby name ni ...,0
1,3,2101106 602 pm chest portable ap clip clip num...,0
2,4,2191315 420 pm chest portable ap clip clip num...,0
3,5,nnp triage notebb known lastname 6 delivered 4...,0
4,6,2175525 1052 chest preop pa lat clip clip numb...,0
...,...,...,...
52949,99985,sinus rhythm normal ecg since previous tracing...,0
52950,99991,21841227 1135 chest pa lat clip clip number ra...,0
52951,99992,2144710 804 chest port line placement clip cli...,0
52952,99995,2147110 815 carotid series complete clip clip ...,0


In [13]:
batch_size = 1000

num_batches = len(data) // batch_size

def transform_batch(batch):
    reformatted_batch = []
    for _, row in batch.iterrows():
        clinical_notes = row['TEXT']
        
        if row['readmitted'] == 0:
            assistant_text = 'No, the patient will not be readmitted.'
        else:
            assistant_text = 'Yes, the patient will be readmitted.'

        conversation = f"<s>[INST] {clinical_notes} [/INST] {assistant_text} </s>"
        reformatted_batch.append(conversation)
    
    return pd.Series(reformatted_batch)

transformed_dataset = []

for batch_idx in range(num_batches):
    start = batch_idx * batch_size
    end = start + batch_size
    batch = data.iloc[start:end]
    transformed_batch = transform_batch(batch)
    transformed_dataset.extend(transformed_batch)

remaining_data = data.iloc[num_batches * batch_size:]
if len(remaining_data) > 0:
    transformed_remaining = transform_batch(remaining_data)
    transformed_dataset.extend(transformed_remaining)

transformed_dataset = pd.DataFrame(transformed_dataset)

transformed_dataset.columns = ['question_prompt_answer']

In [14]:
transformed_dataset

,question_prompt_answer
0,<s>[INST] neonatology attending triage notebab...
1,<s>[INST] 2101106 602 pm chest portable ap cli...
2,<s>[INST] 2191315 420 pm chest portable ap cli...
3,<s>[INST] nnp triage notebb known lastname 6 d...
4,<s>[INST] 2175525 1052 chest preop pa lat clip...
...,...
41960,<s>[INST] sinus rhythm normal ecg since previo...
41961,<s>[INST] 21841227 1135 chest pa lat clip clip...
41962,<s>[INST] 2144710 804 chest port line placemen...
41963,<s>[INST] 2147110 815 carotid series complete ...


In [15]:
transformed_dataset.iloc[0,0]

'<s>[INST] neonatology attending triage notebaby name ni 1 known lastname 2 term male infant admitted nicu sepsis evaluation asked evaluate baby dr first name4 namepattern1 3 last name namepattern1 4mother 34 years old g1 p01pns pos ab neg hbsag neg rpr nr ri gbs negpregnancy uncomplicateddelivery csection failure progress apgars 99mother treated antibiotics maternal temp 1003 prior delivery mothers temp lower 2 hours rose 102pe baby name2 ni 5 vigorous agavs 985 hr 145 rr 38 bp 7235 48 o2 sat 100 raheent af soft flat molding notedpalate intactresp breath sounds clear equalcvs s1 s2 normal 318 high pitched systolic murmur lsbabd soft normal bowel sounds organomegalygu normal male testes descended bilaterallyneuro tone wnl symmetrical examds 72assessmentplanterm male infant increased risk sepsiswill check cbc diff plats blood culturewill cover antibiotics least 48 hours pending results culturesfurther work possible lp culture positive clinical signs sepsis develop nursing transfer notep

In [16]:
transformed_dataset_100 = transformed_dataset.sample(n=100)

In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming transformed_dataset is a DataFrame
transformed_train_df, transformed_test_val_df = train_test_split(transformed_dataset, test_size=0.3, random_state=42)
transformed_val_df, transformed_test_df = train_test_split(transformed_test_val_df, test_size=0.5, random_state=42)

In [17]:
# Concatenate all the text data from the CSV file into a single string
text_content = " ".join(transformed_train_df["question_prompt_answer"].astype(str))

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Ensure text_content is a string
if isinstance(text_content, list):
    text_content = "".join(text_content)

# Define the parameters for text splitting
chunk_size = 1000  # Size of each text chunk
chunk_overlap = 20  # Overlap between adjacent text chunks

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Split the extracted data into text chunks
text_chunks = text_splitter.split_text(text_content)

# Print the text chunks
for i, chunk in enumerate(text_chunks):
    print(f"Text Chunk {i + 1}:")
    print(chunk)
    print("=" * 50)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [19]:
len(text_chunks)

4615

In [20]:
#get the third chunk
text_chunks[2]

'doctor first name 73 2110710 358 pm radline telephonefax 1 249 radiology consult service hear preliminary results prior transcription call radiology listen line telephonefax 1 250 2110714 354 pm chest preop pa lat clip clip number radiology 81444 reason ro chf infiltrate hospital 2 medical condition 71 year old man cad elec cabg ath lab last name lf first name3 lf 330 pm reason examination ro chf infiltrate final report reason examination preoperative evaluation patient cabg pa lateral upright chest radiographs reviewed prior films available comparison heart size normal mediastinum normal position width shape lungs clear pleural surfaces smooth pleural effusion impression evidence acute cardiopulmonary process patienttest informationindication left ventricular functionheight 74weight lb 215bsa m2 224 m2bp mm hg 13058status outpatientdatetime 2110716 0800test tte completedoppler full doppler color dopplercontrast nonetechnical quality adequateinterpretationfindingsleft atrium elongated

In [21]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
class TextDocument:
    def __init__(self, page_content):
        self.page_content = page_content
        self.metadata = {}  # Initialize metadata as an empty dictionary

In [23]:
# Define the TextDocument class with metadata attribute
class TextDocument:
    def __init__(self, page_content):
        self.page_content = page_content
        self.metadata = {}  # Initialize metadata as an empty dictionary

# Convert text chunks into TextDocument objects
documents = [TextDocument(page_content=chunk) for chunk in text_chunks]

# Create embeddings and store them using FAISS
vector_store = FAISS.from_documents(documents, embedding=embeddings)

In [6]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: mistralai/Mistral-7B-Instruct-v0.1. Received error Model path does not exist: mistralai/Mistral-7B-Instruct-v0.1 (type=value_error)

In [25]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [30]:
query = " karthik is suffering from viral fever from past week; and he had been discharged from hospital within 3 days; Will he get readmitted or not?"

In [31]:
qa.run(query)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     483.17 ms
llama_print_timings:      sample time =       0.12 ms /     1 runs   (    0.12 ms per token,  8547.01 tokens per second)
llama_print_timings: prompt eval time =   10291.14 ms /   316 tokens (   32.57 ms per token,    30.71 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   10343.76 ms /   317 tokens


''

In [32]:
query1 = " A person is suffering from viral fever from past week; his upright chest radiographs are reviewed and it is serious;Will he get readmitted or not?"

In [33]:
qa.run(query1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     483.17 ms
llama_print_timings:      sample time =      10.33 ms /    89 runs   (    0.12 ms per token,  8615.68 tokens per second)
llama_print_timings: prompt eval time =   22125.90 ms /   589 tokens (   37.57 ms per token,    26.62 tokens per second)
llama_print_timings:        eval time =   13990.63 ms /    88 runs   (  158.98 ms per token,     6.29 tokens per second)
llama_print_timings:       total time =   36375.56 ms /   677 tokens


" No, the patient will not be readmitted. The chest X-ray shows evidence of pneumonia, but there is no indication that the person's condition has worsened since the last evaluation. The radiology findings are consistent with a previous study and the final report indicates that there is no evidence of progression or change. Therefore, it seems unlikely that the patient will be readmitted to the hospital based on these findings."

In [37]:
query2 = " A person is suffering from viral fever from past week; his upright chest radiographs are reviewed and it is not serious;Will he get readmitted or not?"

In [38]:
qa.run(query2)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     483.17 ms
llama_print_timings:      sample time =       1.26 ms /    11 runs   (    0.11 ms per token,  8744.04 tokens per second)
llama_print_timings: prompt eval time =     482.68 ms /    16 tokens (   30.17 ms per token,    33.15 tokens per second)
llama_print_timings:        eval time =    1338.93 ms /    10 runs   (  133.89 ms per token,     7.47 tokens per second)
llama_print_timings:       total time =    1842.37 ms /    26 tokens


' No, the patient will not be readmitted.'